In [ ]:
from datacube.utils.dask import start_local_dask
from dep_tools.loaders import LandsatOdcLoader
from dep_tools.namers import LocalPath
from dep_tools.writers import LocalDsWriter

from src.run_task import GeoMADLandsatProcessor, get_grid

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Optionally set up a local dask cluster
client = start_local_dask()
client.dashboard_link

In [ ]:
# Study site configuration
region_code = "66,19"

# Issues
# Cloudy island
# https://deppcpublicstorage.blob.core.windows.net/output/dep_ls_geomad/0-1-0/099/022/2023/dep_ls_geomad_099_022_2023.stac-item.json
region_code = "99,22"
# Noisy tile
# https://deppcpublicstorage.blob.core.windows.net/output/dep_ls_geomad/0-1-0/063/020/2023/dep_ls_geomad_063_020_2023.stac-item.json
region_code="63,20"


datetime = "2023"
item_id = region_code

# And get the study site
grid = get_grid()
cell = grid.loc[[(region_code)]]
cell.explore()

In [ ]:
# Set up a data loader
loader = LandsatOdcLoader(
    epsg=3832,
    datetime=datetime,
    dask_chunksize=dict(time=1, x=4096, y=4096),
    odc_load_kwargs=dict(
        fail_on_error=False,
        resolution=30,
        # bands=["qa_pixel", "red", "green", "blue", "nir08", "swir16", "swir22"],
        groupby="solar_day",
        bands=["qa_pixel", "red", "green", "blue"]
    ),
    exclude_platforms=["landsat-7"],
    nodata_value=0,
    keep_ints=True,
    load_as_dataset=True,
    only_tier_one=True,
    fall_back_to_tier_two=True
)

# Run the load process, which is lazy-loaded
input_data = loader.load(cell)
input_data

In [ ]:
# Set up a data processor
processor = GeoMADLandsatProcessor(
    scale_and_offset=False,
    geomad_options=dict(
        work_chunks=(601, 601),
        num_threads=10,
    ),
    filters=[("closing", 5), ("opening", 5)],
    keep_ints=True
)

# Plan the processing. Actually loads the data.
output_data = processor.process(input_data)
output_data

In [ ]:
output_data[["red", "green", "blue"]].squeeze().to_array().plot.imshow(size=10, vmin=7000, vmax=10000)

In [ ]:
output_data["count"].plot.imshow(size=10)

In [ ]:
# Testing the Azure writer

# from dep_tools.writers import AzureDsWriter
# from dep_tools.namers import DepItemPath

# itempath = DepItemPath("geomad", "test", "0.0", datetime)

# writer = AzureDsWriter(
#     itempath=itempath,
#     overwrite=True,
#     convert_to_int16=False,
#     extra_attrs=dict(dep_version="0.0"),
# )

# writer.write(output_data, "test")


In [ ]:
from odc.stac import load
from pystac import Item

item = Item.from_file("https://deppcpublicstorage.blob.core.windows.net/output/dep_geomad_test/0-0/test/2023-01/dep_geomad_test_test_2023-01.stac-item.json")

data = load([item], chunks={})
data

In [ ]:
data.red.isel(time=0).plot.imshow(size=8, robust=True)

In [ ]:
# This is the target path
dep_path = LocalPath(
    local_folder="data",
    sensor="ls",
    dataset_id="geomad",
    version="0.0.0",
    time=datetime,
    zero_pad_numbers=True
)

# Set up a writer and write out the files
writer = LocalDsWriter(
    itempath=dep_path,
    output_nodata=0,
    use_odc_writer=True,
    overwrite=True,
    convert_to_int16=False
)
print(f"Writing to: {dep_path._folder(item_id)}")
out_files = writer.write(loaded, item_id)

In [ ]:
# Make sure written files are readable
stac_path = writer.itempath.path(item_id, ext=".stac-item.json")

item = Item.from_file(stac_path)
item.validate()